#  Human Voice

## Reference

In [ ]:
# https://www.kaggle.com/datasets/alarmanovi/gender-recognition-by-voice-2023

## Impor Libraries

In [5]:
import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gri
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report



## Import Data

In [10]:
# Dataset Column Overview: 
# ------------------------------------------------------------------------------

# X01: Mean Frequency (mean_freq): Computed using Fast Fourier Transform (FFT). Represents the average frequency of the audio signal.
# X02: Standard Deviation of Frequency (sd_freq): Calculated using the standard deviation of the spectral centroid. Indicates the spread or variability of the spectral centroid.
# X03: Median Frequency (median_freq): Obtained from the median of the spectral centroid. Represents the middle value of the distribution of spectral centroid frequencies.
# X04: 25th Percentile Frequency (q25_freq): Quantiles of the spectral centroid. Provide information about the distribution of spectral centroid frequencies.
# X05: 75th Percentile Frequency (q75_freq): Quantiles of the spectral centroid. Provide information about the distribution of spectral centroid frequencies.
# X06: Interquartile Range of Frequency (iqr_freq): The range between the 25th and 75th percentiles. Measures the spread of the middle 50% of spectral centroid frequencies.
# X07: Skewness (skewness): Descriptive statistics of the spectral bandwidth. Indicate the shape of the spectral bandwidth distribution.
# X08: Kurtosis (kurtosis_val): Descriptive statistics of the spectral bandwidth. Indicate the shape of the spectral bandwidth distribution.
# X09: Spectral Flatness (spectral_flatness): Represents the flatness of the power spectrum. A measure of how noise-like or tonal the audio is.
# X10: tempogram: a mid-level representation of tempo information, is constructed to characterize tempo variation and local pulse in the audio signal.
# X11: Mode Frequency (mode_freq): Extracted from the tempogram. Represents the mode (most common) frequency in the audio signal.
# X12: Centroid Frequency (centroid_freq): The mean of the spectral centroid. Provides the center of mass of the spectral distribution.
# X13: Peak Frequency (peak_freq): The frequency with the highest amplitude in the spectral centroid. Indicates the dominant frequency in the audio signal.
# X14: mean_fun: Additional Features Related to Fundamental Frequency. Extracted from the root mean square (RMS) values. Provide information about the fundamental frequency.
# X15: min_fun: Additional Features Related to Fundamental Frequency. Extracted from the root mean square (RMS) values. Provide information about the fundamental frequency.
# X16: max_fun: Additional Features Related to Fundamental Frequency. Extracted from the root mean square (RMS) values. Provide information about the fundamental frequency.
# X17: mean_dom: Additional Features Related to Dominant Frequency. Derived from the tempogram. Offer insights into the dominant frequencies and their range.
# X18: min_dom: Additional Features Related to Dominant Frequency. Derived from the tempogram. Offer insights into the dominant frequencies and their range.
# X19: max_dom: Additional Features Related to Dominant Frequency. Derived from the tempogram. Offer insights into the dominant frequencies and their range.
# X20: dom_range: Additional Features Related to Dominant Frequency. Derived from the tempogram. Offer insights into the dominant frequencies and their range.
# X21: Modulation Index (mod_index): Calculated using the YIN algorithm. Indicates the degree of modulation in the audio signal.
#   y: Label (label): Assigned as 'female' or 'Male' for all processed audio files.


path = 'https://raw.githubusercontent.com/notfakearcher/julian/main/02_data/male_female_voice.csv'
voice = pd.read_csv(path)

In [9]:
voice.isna().sum()

mean_freq            0
sd_freq              0
median_freq          0
q25_freq             0
q75_freq             0
iqr_freq             0
skewness             0
kurtosis_val         0
spectral_flatness    0
tempogram            0
mode_freq            0
centroid_freq        0
peak_freq            0
mean_fun             0
min_fun              0
max_fun              0
mean_dom             0
min_dom              0
max_dom              0
fund_freq            0
mod_index            0
label                0
dtype: int64